# Extracting coastlines from Global Coastal Transect System GCTS and Satellite Derived Shorelines System SDSS

Run the first few cells to load required functions and jump to the section you're interested in afterwards. 

In [ ]:
import sys

# sys.path.insert(0, "..\src")
sys.path.insert(0, "../src")

import dask
# NOTE: query planning is not implemented in dask_geopandas yet, so we have to set 
# it to False before we do any dask_geopandas import 
dask.config.set({"dataframe.query-planning": False})

from coastlines4shorelines.utils import transect_origins_to_coastline,retrieve_transects_by_roi,shoreline_intersections_to_coastline,filter_sp

import logging
import os
import pathlib


from dask.dataframe.utils import make_meta

import dask_geopandas
import duckdb
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastmonitor.geo.geometries import geo_bbox

load_dotenv(override=True)

sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
storage_options = {"account_name": account_name, "credential": sas_token}

logging.getLogger("azure").setLevel(logging.WARNING)

## Load from STAC catalog

Load the transects from our CoCliCo STAC catalog. 

In [ ]:
coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)

In [ ]:
coclico_catalog

In [ ]:
list(coclico_catalog.get_all_collections())

In [ ]:
gcts = coclico_catalog.get_child("gcts")
gcts

### Define a region of interest (ROI) based on a kml file

In [ ]:
import fiona

fiona.drvsupport.supported_drivers["KML"] = "rw"
kml_fp = pathlib.Path(r"d:\FHICS\ShorelineS\ROIs\North_Carolina_Virginia.kml")
roi = gpd.read_file(kml_fp, driver="KML")

### Get all transects for this region of interest

In [ ]:
transects_roi = retrieve_transects_by_roi(roi, storage_options=storage_options)

In [ ]:
transects_roi.head()

In [ ]:
list(transects_roi)

### Apply transect_origins_to_coastline and explode to get one LineString per row

In [ ]:
coastline = (
    transects_roi.groupby("coastline_id")
    .apply(transect_origins_to_coastline)
    .explode()
    .reset_index(name="geometry")
    .drop(columns=["level_1"])
)
coastline.head()

In [ ]:
coastline = gpd.GeoDataFrame(coastline, crs=4326)
coastline = gpd.overlay(coastline, roi[["geometry"]]).explode(index_parts=False)
coastline.head()

### Explore the ROI polygon and the cleaned up base coastline

In [ ]:
m = roi.explore()
gpd.GeoDataFrame(coastline, crs=4326).explore(color="red", m=m)

### Load ShorelineMonitor SDS series 

In [ ]:
sdss = dask_geopandas.read_parquet(
    "az://shorelinemonitor-raw-series/release/2024-04-15/sp_NC.parquet",
    storage_options=storage_options,
).compute()
sdss = sdss.assign(time=pd.to_datetime(sdss.time).dt.strftime("%Y-%m-%d"))

### Implement the filtering function to raw time series

In [ ]:
sdss_clean = filter_sp(sdss)
list(sdss_clean)

### Sort on transect name and extract transect name into coastline_id, segment_id and transect_id

In [ ]:
    sdss = sdss_clean.sort_values("tr_name")
    sdss[["coastline_id", "segment_id", "transect_id"]] = (
        sdss.tr_name.str.extract(r"cl(\d+)s(\d+)tr(\d+)")
    )
    sdss = sdss.astype(
        {"coastline_id": int, "segment_id": int, "transect_id": int}
    )
    sdss.head()

### Create subsets of sdss for 2010 and 2020

In [ ]:
import pandas as pd
# Ensure your time column is in datetime format
sdss['time'] = pd.to_datetime(sdss['time'])

# Define the specific date
specific_date = pd.Timestamp('2010-01-01')

# Filter the GeoDataFrame
sdss_2010 = sdss.loc[sdss['time'] == specific_date]
m=sdss_2010.geometry.explore()

In [ ]:
sdss_2010

### Try a merge of the OSM smoothed coastline and the points for specific years from sdss 

In [ ]:
merge_2010=pd.merge(transects_roi[["tr_name","lon","lat"]],sdss_2010,how="left",on="tr_name")

In [ ]:
c_2010=gpd.GeoDataFrame(merge_2010,geometry=gpd.GeoSeries.from_xy(merge_2010.lon_y,merge_2010.lat_y,crs=4326))
c_2010=c_2010.rename(columns={"lon_y":"lon","lat_y":"lat"})
coast_2010 = shoreline_intersections_to_coastline(c_2010)
coast_2010
gpd.GeoDataFrame(geometry=coast_2010,crs=4326).explore()

### Show both datasets; notice they are points

In [ ]:
import pandas as pd
# Ensure your time column is in datetime format
sdss['time'] = pd.to_datetime(sdss['time'])

# Define the specific date
specific_date = pd.Timestamp('2020-01-01')

# Filter the GeoDataFrame
sdss_2020 = sdss.loc[sdss['time'] == specific_date]
sdss_2020.geometry.explore(color="red", m=m)

In [ ]:
import pandas as pd
# Ensure your time column is in datetime format
sdss['time'] = pd.to_datetime(sdss['time'])

# Define the specific date
specific_date = pd.Timestamp('2022-01-01')

# Filter the GeoDataFrame
sdss_2022 = sdss.loc[sdss['time'] == specific_date]
sdss_2022.geometry.explore(color="green", m=m)

In [ ]:
#list(sdss_2010)

### Now connect the dots

In [ ]:
coastline_2010 = shoreline_intersections_to_coastline(sdss_2010)
coastline_2020 = shoreline_intersections_to_coastline(sdss_2020)
#list(coastline_2020)

### Turn the LineStrings into a GeoDataFrame

In [ ]:
coastline_2010 = gpd.GeoDataFrame(geometry=coastline_2010,crs=4326)
coastline_2020 = gpd.GeoDataFrame(geometry=coastline_2020,crs=4326)

### Now explore both coastlines

In [ ]:
m = coastline_2010.explore()
coastline_2020.explore(color="red", columnn="time", m=m)